# Analysing Nashville Police Activity Data

## Does the gender affect if someone gets stopped by the police?

I looked at analysing which gender was stopped more, the reason and which outcomes. 

In [1]:
# Import the packages that I would find useful for this analysis 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from datetime import date

%matplotlib inline

In [2]:
# Import the dataset retrieved from https://openpolicing.stanford.edu/data/ 
# Merge the columns date and time and drop the two columns 
# Show the first 5 rows of the data set

nash_df = pd.read_csv("tn_nashville_2020_04_01.csv")
nash_df['datetime'] = pd.to_datetime(nash_df['date'] + ' ' + nash_df['time'])
nash_df = nash_df.drop(columns = ['date', 'time'], axis = 1)
print(nash_df.head())

/var/folders/jr/q9y2nm4s7fb_r6mkzg214rvm0000gn/T/ipykernel_47895/2519848530.py:5: DtypeWarning: Columns (6,8,15,16,17,22,23,24,25,29,30,31,32,33,35,36,37,38,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  nash_df = pd.read_csv("tn_nashville_2020_04_01.csv")


  raw_row_number                                           location   
0         232947  DOMINICAN DR & ROSA L PARKS BLVD, NASHVILLE, T...  \
1         237161            1122 LEBANON PIKE, NASHVILLE, TN, 37210   
2         232902                       898 DAVIDSON DR, , TN, 37205   
3         233219  MURFREESBORO PIKE & NASHBORO BLVD, ANTIOCH, TN...   
4         232780                      BUCHANAN ST, NORTH, TN, 37208   

         lat        lng precinct  reporting_area   zone  subject_age   
0  36.187925 -86.798519      6.0          4403.0  611.0         27.0  \
1  36.155521 -86.735902      5.0          9035.0  513.0         18.0   
2  36.117420 -86.895593      1.0          5005.0  121.0         52.0   
3  36.086799 -86.648581      3.0          8891.0  325.0         25.0   
4  36.180038 -86.809109      NaN             NaN    NaN         21.0   

  subject_race subject_sex  ... raw_misd_state_citation_issued   
0        black        male  ...                            NaN  \
1       

I looked into more information of on the data type.
- Info was to show the data types of the rows in the colunmns 
- Describe shows the basic stats of the numerical data 
- dtype.values_counts counts the number of columns with the same data types 

In [3]:
print(nash_df.info())
print(nash_df.describe())
print(nash_df.dtypes.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3092351 entries, 0 to 3092350
Data columns (total 41 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   raw_row_number                  object        
 1   location                        object        
 2   lat                             float64       
 3   lng                             float64       
 4   precinct                        object        
 5   reporting_area                  float64       
 6   zone                            object        
 7   subject_age                     float64       
 8   subject_race                    object        
 9   subject_sex                     object        
 10  officer_id_hash                 object        
 11  type                            object        
 12  violation                       object        
 13  arrest_made                     object        
 14  citation_issued                 object        
 15

Due to the question I wanted to answer, I didn't need all the columns provided. I dropped the the following columns:
- 'raw_row_number' 
- 'lat', 'lng', 'precinct', 'date','time','raw_misd_state_citation_issued',
- 'reporting_area', 'zone','officer_id_hash', 'outcome', 'contraband_found',
- 'contraband_drugs', 'contraband_weapons', 'frisk_performed',
- 'search_conducted', 'search_person', 'search_vehicle', 'search_basis','notes',
- 'raw_suspect_ethnicity', 'raw_driver_searched',
- 'raw_passenger_searched', 'raw_search_consent', 'raw_search_arrest',
- 'raw_search_warrant', 'raw_search_inventory', 'raw_search_plain_view'

In [4]:
nash_df = nash_df.drop(columns = ['raw_row_number', 'lat', 'lng', 'precinct',
                                 'reporting_area', 'zone','officer_id_hash', 'outcome', 'contraband_found',
                                 'contraband_drugs', 'contraband_weapons', 'frisk_performed',
                                 'search_conducted', 'search_person', 'search_vehicle', 'search_basis','notes',
                                 'raw_suspect_ethnicity', 'raw_driver_searched',
                                 'raw_passenger_searched', 'raw_search_consent', 'raw_search_arrest',
                                 'raw_search_warrant', 'raw_search_inventory', 'raw_search_plain_view', 
                                   'raw_misd_state_citation_issued'],axis = 1)

I looked for any missing data present in the data set. The column with the most missing information was raw_written_warning_issued with about 16% missing. Since it was very low I decided to fill the missing categorical information and drop the numerical which was even lower.


In [5]:
nash_df.isnull().sum().sort_values(ascending = False)/len(nash_df)

raw_written_warning_issued     0.159455
vehicle_registration_state     0.010281
subject_sex                    0.004146
violation                      0.002593
reason_for_stop                0.002593
datetime                       0.001768
subject_race                   0.000598
subject_age                    0.000271
warning_issued                 0.000109
raw_verbal_warning_issued      0.000109
raw_traffic_citation_issued    0.000104
citation_issued                0.000103
arrest_made                    0.000009
location                       0.000000
type                           0.000000
dtype: float64

In [6]:
# Separated the data types into numerical and categorical groups 
numerical = nash_df.columns[nash_df.dtypes != "object"]
categorical = nash_df.columns[nash_df.dtypes == "object"]
print(numerical)
print(categorical)

Index(['subject_age', 'datetime'], dtype='object')
Index(['location', 'subject_race', 'subject_sex', 'type', 'violation',
       'arrest_made', 'citation_issued', 'warning_issued', 'reason_for_stop',
       'vehicle_registration_state', 'raw_verbal_warning_issued',
       'raw_written_warning_issued', 'raw_traffic_citation_issued'],
      dtype='object')


Found and filled all the categorical data

In [7]:
nash_df[categorical].isnull().sum().sort_values(ascending = False)/len(nash_df)

raw_written_warning_issued     0.159455
vehicle_registration_state     0.010281
subject_sex                    0.004146
violation                      0.002593
reason_for_stop                0.002593
subject_race                   0.000598
warning_issued                 0.000109
raw_verbal_warning_issued      0.000109
raw_traffic_citation_issued    0.000104
citation_issued                0.000103
arrest_made                    0.000009
location                       0.000000
type                           0.000000
dtype: float64

In [8]:
nash_df[categorical] = nash_df[categorical].fillna("No Information")

In [10]:
nash_df[categorical].isnull().sum().sort_values(ascending = False)/len(nash_df)

location                       0.0
subject_race                   0.0
subject_sex                    0.0
type                           0.0
violation                      0.0
arrest_made                    0.0
citation_issued                0.0
warning_issued                 0.0
reason_for_stop                0.0
vehicle_registration_state     0.0
raw_verbal_warning_issued      0.0
raw_written_warning_issued     0.0
raw_traffic_citation_issued    0.0
dtype: float64

Found and filled all the numerical data

In [11]:
nash_df[numerical].isnull().sum().sort_values(ascending = False)/ len(nash_df)

datetime       0.001768
subject_age    0.000271
dtype: float64

In [12]:
nash_df.dropna(subset=['datetime', 'subject_age'], how='any', inplace=True)

In [13]:
nash_df[numerical].isnull().sum().sort_values(ascending = False)/ len(nash_df)

subject_age    0.0
datetime       0.0
dtype: float64

In [14]:
# Look at the different options in the categorical data and the amount of times they occ
print(nash_df['subject_sex'].value_counts())
print(nash_df['type'].value_counts())
print(nash_df['violation'].value_counts())

subject_sex
male              1823297
female            1250216
No Information      12535
Name: count, dtype: int64
type
vehicular    3086048
Name: count, dtype: int64
violation
moving traffic violation       1543806
vehicle equipment violation     993995
safety violation                185765
registration                    185534
seatbelt violation              103133
investigative stop               56350
parking violation                 8422
No Information                    7934
child restraint                   1109
Name: count, dtype: int64


This is to show the percentage of males and females recorded
- 59.08% is male
- 40.51% is female
- 0.41% is no information 

In [15]:
print((nash_df['subject_sex'].value_counts()/nash_df['subject_sex'].count())*100)

subject_sex
male              59.081939
female            40.511878
No Information     0.406183
Name: count, dtype: float64


### Which gender committed the most violations?
- Groupby was used to group the 'subject_sex'based on 'violation' and count the amount of violations made
    - Males committed the most violations in Nashville according to the record 

In [16]:
violation_by_gender = nash_df.groupby(['subject_sex']).agg({'violation': ['count']})
violation_by_gender.columns = ['violation_count']
violation_by_gender = violation_by_gender.reset_index()
print(violation_by_gender)

      subject_sex  violation_count
0  No Information            12535
1          female          1250216
2            male          1823297


### Which violation was committed the most from each genders?
- Based on grouping the 'subject_sex'and 'violation' columns and count the each of the violations 
    - female     moving traffic violation           631911
    - male     moving traffic violation           906016
    - No Information     moving traffic violation             5879

In [17]:
violation_by_group = nash_df.groupby(['subject_sex', 'violation']).agg({'violation': ['count']})
violation_by_group.columns = ['violation_count']
violation_by_group = violation_by_group.reset_index()
print(violation_by_group)

       subject_sex                    violation  violation_count
0   No Information               No Information              160
1   No Information              child restraint                3
2   No Information           investigative stop              310
3   No Information     moving traffic violation             5879
4   No Information            parking violation              685
5   No Information                 registration              805
6   No Information             safety violation              920
7   No Information           seatbelt violation              350
8   No Information  vehicle equipment violation             3423
9           female               No Information             3019
10          female              child restraint              720
11          female           investigative stop            19229
12          female     moving traffic violation           631911
13          female            parking violation             2986
14          female       

In [18]:
# Showing which gender committed the most violations and which violation they did the most.
violation_by_group2 = violation_by_group.loc[violation_by_group['violation_count'].idxmax()]
print(violation_by_group2)

subject_sex                            male
violation          moving traffic violation
violation_count                      906016
Name: 21, dtype: object


In [19]:
# Show the count of the violations committed the most for each of the genders.
violation_by_group.groupby('subject_sex')['violation_count'].max().reset_index().sort_values(['violation_count'], ascending = False) 

,subject_sex,violation_count
2,male,906016
1,female,631911
0,No Information,5879


### Which gender was given the most arrest, warning and citations
Based on the analysis:
- Males were given more of each outcomes.
- Warnings were given out more than the other outcomes, second was citation then lastly arrests
- Males recieved more citations and arrests than women, while women recieved more warnings 

In [20]:
warning = nash_df[nash_df['warning_issued'] == True]
print('Number of males and females that receieved warnings','\n', warning['subject_sex'].value_counts().sort_values(ascending = False))

male_warned_percentage = (len(nash_df[(nash_df['subject_sex']=='male') & (nash_df['warning_issued']== True)])/len(nash_df[nash_df['subject_sex']=='male'])) * 100
female_warned_percentage = (len(nash_df[(nash_df['subject_sex']=='female') & (nash_df['warning_issued']== True)])/len(nash_df[nash_df['subject_sex']=='female'])) * 100

print('Percentage of males recorded that recieved warnings',male_warned_percentage)
print('Percentage of females recorded that recieved warnings',female_warned_percentage)

Number of males and females that receieved warnings 
 subject_sex
male              1440990
female            1005605
No Information      10100
Name: count, dtype: int64
Percentage of males recorded that recieved warnings 79.03210502732139
Percentage of females recorded that recieved warnings 80.43450091824133


There is more males that was given warnings (overall), 79.03% of the recorded males were given warnings while there is only 80.43% of the recorded females.

In [21]:
citation = nash_df[nash_df['citation_issued'] == True]
print('Number of males and females that receieved citations\n', citation['subject_sex'].value_counts().sort_values(ascending = False))

male_citation_percentage = (len(nash_df[(nash_df['subject_sex']=='male') & (nash_df['citation_issued']== True)])/len(nash_df[nash_df['subject_sex']=='male'])) * 100
female_citation_percentage = (len(nash_df[(nash_df['subject_sex']=='female') & (nash_df['citation_issued']== True)])/len(nash_df[nash_df['subject_sex']=='female'])) * 100

print('Percentage of males recorded that recieved citations',male_citation_percentage)
print('Percentage of females recorded that recieved citations',female_citation_percentage)

Number of males and females that receieved citations
 subject_sex
male              422347
female            269790
No Information      1090
Name: count, dtype: int64
Percentage of males recorded that recieved citations 23.163916794685672
Percentage of females recorded that recieved citations 21.579471067399552


There is more males that was given citations (overall), 23% of the recorded males were given citations while there is only 21.6% of the recorded females.


In [22]:
arrested = nash_df[nash_df['arrest_made'] == True]
print('Number of males and females that was arrested\n', arrested['subject_sex'].value_counts().sort_values(ascending = False))

male_arrest_percentage = (len(nash_df[(nash_df['subject_sex']=='male') & (nash_df['arrest_made']== True)])/len(nash_df[nash_df['subject_sex']=='male'])) * 100
female_arrest_percentage = (len(nash_df[(nash_df['subject_sex']=='female') & (nash_df['arrest_made']== True)])/len(nash_df[nash_df['subject_sex']=='female'])) * 100

print('Percentage of males recorded that was arrested',male_arrest_percentage)
print('Percentage of females recorded that was arrested',female_arrest_percentage)

Number of males and females that was arrested
 subject_sex
male              36559
female            13330
No Information      178
Name: count, dtype: int64
Percentage of males recorded that was arrested 2.0051039408280715
Percentage of females recorded that was arrested 1.066215757917032


There is more males that were arrested (overall), 2.01% of the recorded males were arrested while there is only 1.07% of the recorded females.

# Conclusion

- Overall we can see that most of the observed stops were to men than women. The most common violation was due to moving traffic violation in both genders.
- Males recieved more of each outcome compared to women 
- Majority of men and women stopped only recieved a warning, secondly was citations then arrest 
- In each of the outcomes, majority of the population of women were given warnings which was higher than the population of men recorded, while more men were given citations and arrested.

# Future Work

- Use data visualisation tool to look more at data provided 